This is my python file for my model

In [ ]:
# Mounting my Google Drive
# comment out if using on VsCode
from google.colab import drive
import os

mountPath = '/content/drive'
drive.mount(mountPath, force_remount=True)#Forcing remount if needed

Mounted at /content/drive


# Chapter 1 - downloading and altering the first Dataset

In [ ]:
#cloning my repo from github
!git clone https://github.com/Unbreakable60000/DementiaDetection.git

# Defining the source folder in the repo
sourceFolder = "/content/DementiaDetection/datasetFolder/TIHMDataset/Dataset"

# Defining the destination folder in your Drive
destFolder = "/content/drive/MyDrive/synoptic/oldDataset"

# Making sure the destination exists (it's REAL)
import os
os.makedirs(destFolder, exist_ok=True)

# Copying all files from the repo folder to Drive
import shutil
for filename in os.listdir(sourceFolder):
    full_file_name = os.path.join(sourceFolder, filename)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, destFolder)

# Listing all files in my Drive folder to confirm
!ls "{dest_folder}"

fatal: destination path 'DementiaDetection' already exists and is not an empty directory.
ls: cannot access '{dest_folder}': No such file or directory


Dataset before alteration can be accessed here:
https://zenodo.org/records/7622128

In [ ]:
import pandas as pd

activityDF = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Activity.csv")
demographicsDF = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Demographics.csv")
labelsDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Labels.csv")
physiologyDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Physiology.csv")
sleepDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Sleep.csv")

dfItems = {"activityDf": activityDF,"demographicsDF": demographicsDF,"labelsDf": labelsDf,"physiologyDf": physiologyDf, "sleepDf":sleepDf}

In [ ]:
for name,df in dfItems.items():#itterates through list and gives column names and how many unique values are in each
    print(f"{name}:")
    print("--------")
    print(df.columns.tolist())
    for col in df.columns:
        num_unique = df[col].nunique()
        print(f"{col}: {num_unique} unique values")
    print()

demographicsDF:
--------
['patient_id', 'age', 'sex']
patient_id: 56 unique values
age: 3 unique values
sex: 2 unique values

labelsDf:
--------
['patient_id', 'Agitation', 'Blood pressure', 'Body temperature', 'Body water', 'Pulse', 'Weight']
patient_id: 49 unique values
Agitation: 2 unique values
Blood pressure: 2 unique values
Body temperature: 2 unique values
Body water: 2 unique values
Pulse: 2 unique values
Weight: 2 unique values

physiologyDf:
--------
['patient_id', 'Body Temperature', 'Body weight', 'Diastolic blood pressure', 'Heart rate', 'O/E - muscle mass', 'Skin Temperature', 'Systolic blood pressure', 'Total body water']
patient_id: 55 unique values
Body Temperature: 55 unique values
Body weight: 46 unique values
Diastolic blood pressure: 54 unique values
Heart rate: 54 unique values
O/E - muscle mass: 45 unique values
Skin Temperature: 13 unique values
Systolic blood pressure: 53 unique values
Total body water: 45 unique values

sleepDf:
--------
['patient_id', 'AWAKE'

The above shows 56 unique patients, I will now join all into one csv document. And then try and find more data to include. The current issue is that there are multiple for some individuals. First I am not including activity as it provided information about the indiduduals movements and thats not an area I am focusing on

In [ ]:
print("dfItem removed:")
dfItems.pop("activityDf")

dfItem removed:


,patient_id,location_name,date
0,0697d,Fridge Door,2019-06-28 13:03:29
1,0697d,Kitchen,2019-06-28 13:11:44
2,0697d,Front Door,2019-06-28 13:13:50
3,0697d,Bedroom,2019-06-28 13:13:53
4,0697d,Fridge Door,2019-06-28 13:14:09
...,...,...,...
1030554,fd100,Hallway,2019-06-30 23:48:50
1030555,fd100,Lounge,2019-06-30 23:49:40
1030556,fd100,Kitchen,2019-06-30 23:50:02
1030557,fd100,Front Door,2019-06-30 23:51:28


Now Activity is removed I can continue cleaning

In [ ]:
#dropping dates as they arent needed
labelsDf = labelsDf.drop(columns=['date'])
physiologyDf = physiologyDf.drop(columns=['date'])
sleepDf = sleepDf.drop(columns=['date'])

In [ ]:
labelsDf.head()

,patient_id,type
0,c55f8,Blood pressure
1,16f4b,Blood pressure
2,16f4b,Agitation
3,ec812,Blood pressure
4,16f4b,Agitation


In [ ]:
#changing labelsdf to have multiple columns with 0 or 1 for each of the type conditions (this will make it easier to merge later)

labelsDf = (
    labelsDf
    .pivot_table(
        index='patient_id',
        columns='type',
        aggfunc='size',
        fill_value=0
    )
    .reset_index()
)
# Converting counts to presence (0/1) - this will be better for my models later on
label_cols = labelsDf.columns.drop('patient_id')
labelsDf[label_cols] = (labelsDf[label_cols] > 0).astype(int)

In [ ]:
labelsDf.head()

type,patient_id,Agitation,Blood pressure,Body temperature,Body water,Pulse,Weight
0,0697d,0,1,0,0,1,0
1,099bc,1,1,0,0,0,0
2,0cda9,1,0,0,1,0,0
3,0d5ef,1,1,0,0,1,0
4,0efe8,0,0,0,1,1,0


Next I have expanded physiology by moving it to long format and grouping by patient_id - naming conventions and labelling will be done after merging

In [ ]:
physiologyDf = (
    physiologyDf
    .pivot_table(
        index='patient_id',
        columns='device_type',
        values='value',
        aggfunc='mean'   # average per patient per device
    )
    .reset_index()
)

In [ ]:
physiologyDf.head()

device_type,patient_id,Body Temperature,Body weight,Diastolic blood pressure,Heart rate,O/E - muscle mass,Skin Temperature,Systolic blood pressure,Total body water
0,0697d,36.386688,86.200000,80.000000,53.400000,64.550000,NaN,156.600000,50.900000
1,099bc,36.769330,52.614000,87.253333,69.226667,37.178378,NaN,151.466667,51.532432
2,0d5ef,36.572138,99.614286,88.266667,80.377778,67.933333,NaN,145.044444,48.633333
3,0efe8,36.370707,70.353731,75.800000,72.733333,39.607895,NaN,135.288889,42.381579
4,0f352,36.286647,72.400000,78.222222,74.000000,NaN,NaN,137.222222,NaN


In [ ]:
sleepDf.head()

,patient_id,state,heart_rate,respiratory_rate,snoring
0,0f352,AWAKE,69.0,14.0,False
1,0f352,AWAKE,66.0,14.0,False
2,0f352,AWAKE,70.0,14.0,False
3,0f352,AWAKE,70.0,13.0,False
4,0f352,AWAKE,68.0,13.0,False


In [ ]:
sleepDf = (
    sleepDf
    .pivot_table(
        index='patient_id',
        columns='state',
        aggfunc='size',
        fill_value=0
    )
    .reset_index()
)

# Converting counts to presence (0/1) - this will be better for my models later on (binary)
state_cols = sleepDf.columns.drop('patient_id')
sleepDf[state_cols] = (sleepDf[state_cols] > 0).astype(int)

In [ ]:
sleepDf.head()

state,patient_id,AWAKE,DEEP,LIGHT,REM
0,0f352,1,1,1,1
1,16f4b,1,1,1,1
2,1fbe4,1,1,1,1
3,30a32,1,1,1,1
4,55cd4,1,1,1,1


In [ ]:
#now all have been aggregated I can merge them

In [ ]:
from functools import reduce #merging the new aggregated files with the demographics "patients" df

dfs = [demographicsDF, labelsDf, physiologyDf, sleepDf]

mergedDf = reduce(
    lambda left, right: pd.merge(left, right, on='patient_id', how='left'),
    dfs
)

In [ ]:
mergedDf.head()

,patient_id,age,sex,Agitation,Blood pressure,Body temperature,Body water,Pulse,Weight,Body Temperature,...,Diastolic blood pressure,Heart rate,O/E - muscle mass,Skin Temperature,Systolic blood pressure,Total body water,AWAKE,DEEP,LIGHT,REM
0,b9d58,"(70, 80]",Female,1.0,0.0,0.0,0.0,0.0,0.0,36.330500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c55f8,"(80, 90]",Female,0.0,1.0,0.0,0.0,1.0,1.0,36.728481,...,82.730000,60.450000,35.636585,34.529511,144.98,47.571951,1.0,1.0,1.0,1.0
2,16f4b,"(80, 90]",Male,1.0,1.0,0.0,0.0,0.0,0.0,36.745277,...,88.785714,78.785714,NaN,34.586048,146.00,NaN,1.0,1.0,1.0,1.0
3,fd100,"(90, 110]",Female,0.0,1.0,0.0,0.0,0.0,0.0,36.627000,...,75.333333,56.333333,37.500000,NaN,148.00,48.700000,NaN,NaN,NaN,NaN
4,1fbe4,"(80, 90]",Male,0.0,1.0,0.0,0.0,1.0,0.0,36.034663,...,78.876923,54.215385,60.618919,33.088047,146.00,51.121622,1.0,1.0,1.0,1.0


In [ ]:
outputPath = "/content/DementiaDetection/datasetFolder/mergedDataset.csv"
mergedDf.to_csv(outputPath, index=False)

# Chapter 2 - Looking into creating a synthetic dataset

## why choose this?

As of limitations with gaining access to data I've decided to look into generating my own dataset with what I need based on field for questions and statistics produced by reputable sites like the NHS or .Gov

## Fields I will have:

1. **patientID** - random hash not repeatable

2. **Age** - 30-100 (decide later exact ages)

3. **Gender** - Male (0) or Female (1) (using binary where possible)

4. **dementiaHistory** - No(0) or Yes(1) - does the patient have a history of dementia in the family
5. **cognitionMedications** - No(0) or Yes(1) - is the patient on medications that could affect cognition
6. **cardiometabolic** - No(0) or Yes(1) - does the patient have conditions like hypertension, diabetes, stroke, or heart disease?
7. Smoked - No(0) or Yes(1) - has the patient ever smoked?
8. Alchohol - low(0) or moderate(1) or good(2) [ordinal datawise] - average alchohol consumption level
9. physical - low(0) or moderate(1) or good(2) [ordinal datawise] - average physical activity level
10. sleep - poor(0) or average(1) or good(2)
11. memory - No (0) or Yes(1) - does the patient often forget things?
12. mentalState - No (0) or Yes(1) - does the patient experience any signs of depression or anxiety









